<a href="https://colab.research.google.com/github/fralfaro/MAT281/blob/main/docs/labs/lab_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# MAT281 - Laboratorio N°03





**Objetivo**: Aplicar técnicas avanzadas de manipulación y análisis de datos con pandas sobre un conjunto real de datos de contenido de Netflix, reforzando buenas prácticas y métodos eficientes sin recurrir a `groupby`, `merge`, `pivot`, ni `join`.



**Dataset**:

Trabajaremos con el archivo `netflix_titles.csv`, que contiene información sobre los títulos disponibles en la plataforma Netflix hasta el año 2021.

| Variable       | Clase     | Descripción                                                                 |
|----------------|-----------|------------------------------------------------------------------------------|
| show_id        | caracter  | Identificador único del título en el catálogo de Netflix.                   |
| type           | caracter  | Tipo de contenido: 'Movie' o 'TV Show'.                                     |
| title          | caracter  | Título del contenido.                                                       |
| director       | caracter  | Nombre del director (puede ser nulo).                                       |
| cast           | caracter  | Lista de actores principales (puede ser nulo).                              |
| country        | caracter  | País o países donde se produjo el contenido.                                |
| date_added     | fecha     | Fecha en la que el título fue agregado al catálogo de Netflix.              |
| release_year   | entero    | Año de lanzamiento original del título.                                     |
| rating         | caracter  | Clasificación por edad (por ejemplo: 'PG-13', 'TV-MA').                      |
| duration       | caracter  | Duración del contenido (minutos o número de temporadas para series).        |
| listed_in      | caracter  | Categorías o géneros en los que está clasificado el contenido.              |
| description    | caracter  | Breve sinopsis del contenido.                                               |




In [18]:
import pandas as pd

# Cargar datos
df = pd.read_csv('https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/netflix_titles.csv')
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...



### Parte 1: Limpieza y preparación

1. Revisar y describir el dataset:

   * ¿Cuántas filas y columnas tiene?
   * ¿Qué tipos de datos hay?
   * ¿Cuántos valores nulos hay por columna?

2. Transformar la columna `date_added` a tipo fecha.

3. Crear columnas auxiliares con `assign`:

   * Año (`year_added`)
   * Mes (`month_added`)



In [19]:
# 1. Revisar y describir el dataset
print("Cantidad de filas y columnas:", df.shape)
print("Tipos de datos presentes:", df.dtypes.unique())
print("\nValores nulos por columna:")
print(df.isna().sum())

# 2. Transformar 'date_added' a tipo fecha
df["date_added"] = pd.to_datetime(df["date_added"], format="%B %d, %Y", errors="coerce")

# 3. Crear columnas auxiliares con assign
df = df.assign(
    year_added = df["date_added"].dt.year,
    month_added = df["date_added"].dt.month
)

Cantidad de filas y columnas: (8807, 12)
Tipos de datos presentes: [dtype('O') dtype('int64')]

Valores nulos por columna:
show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64


## Parte 2: Técnicas avanzadas de pandas

4. Utilizar `.loc` para seleccionar películas (`type == 'Movie'`) que fueron agregadas después del año 2018.

5. Utilizar `str.contains()` y `str.extract()`:

   * Filtrar títulos que contienen la palabra 'love' (sin distinguir mayúsculas/minúsculas).
   * Extraer la duración en minutos para las películas desde la columna `duration`.

6. Aplicar `explode()` sobre la columna `listed_in` para obtener una fila por cada género.

7. Obtener un top 10 de géneros más frecuentes utilizando `value_counts()`.

8. Aplicar `where()` y `mask()` para marcar las películas de más de 120 minutos como contenido largo en una nueva columna.

9. Utilizar `.loc` para filtrar películas que cumplen con:

   * Más de 100 minutos de duración.
   * Rating igual a `'R'`.
   * País igual a `'United States'`.

10. Utilizar `.style` para formatear visualmente el top 10 de películas más largas.

In [20]:
import numpy as np
# 4) .loc: películas agregadas después de 2018
pelis_despues_2018 = df.loc[
    (df["type"] == "Movie") & (df["date_added"].dt.year > 2018)
].copy()

# 5.a) Títulos que contienen 'love' (case-insensitive)
titulos_palabra_love = df.loc[
    df["title"].str.contains(r"\blove\b", case=False, na=False),
    ["show_id", "title", "type", "release_year"]
]

# 5.b) Extraer duración en minutos SOLO para películas
df["minutes"] = np.where(
    df["type"].eq("Movie"),
    pd.to_numeric(df["duration"].str.extract(r"(\d+)")[0], errors="coerce"),
    np.nan
)

# 6) explode() sobre listed_in (géneros)
df_generos = (
    df.assign(genre=df["listed_in"].str.split(r",\s*"))
      .explode("genre")
)

# 7) Top 10 géneros más frecuentes con value_counts()
generos_mas_frecuentes = df_generos["genre"].value_counts().head(10)

# 8) where()/mask(): marcar >120 min como contenido largo
df["is_long"] = df["minutes"].mask(df["minutes"] <= 120).notna()

# 9) .loc filtros combinados
peliculas_us_r_mas100 = df.loc[
    (df["type"] == "Movie")
    & (df["minutes"] > 100)
    & (df["rating"] == "R")
    & (df["country"].fillna("").str.strip() == "United States"),
    ["title", "minutes", "rating", "country", "release_year", "date_added"]
].sort_values("minutes", ascending=False)

conteo_us_r_mas100 = len(peliculas_us_r_mas100)

# 10) .style: formatear el top 10 de películas más largas
peliculas_top10_largas = (
    df.loc[df["type"] == "Movie", ["title", "minutes", "rating", "release_year"]]
      .sort_values("minutes", ascending=False)
      .head(10)
)

styled_top10 = (
    peliculas_top10_largas.style
        .bar(subset=["minutes"])                # barra visual en minutos
        .set_caption("Top 10 películas más largas en el dataset")
        .format({"minutes": "{:.0f}"})
)

print("Películas agregadas después de 2018:", len(pelis_despues_2018))
print("\nTop 10 géneros más frecuentes:\n", generos_mas_frecuentes)
print("Películas 'R' de EE.UU. con >100 min (igualdad exacta):", conteo_us_r_mas100)
styled_top10

Películas agregadas después de 2018: 3701

Top 10 géneros más frecuentes:
 genre
International Movies        2752
Dramas                      2427
Comedies                    1674
International TV Shows      1351
Documentaries                869
Action & Adventure           859
TV Dramas                    763
Independent Movies           756
Children & Family Movies     641
Romantic Movies              616
Name: count, dtype: int64
Películas 'R' de EE.UU. con >100 min (igualdad exacta): 243


,title,minutes,rating,release_year
4253,Black Mirror: Bandersnatch,312,TV-MA,2018
717,Headspace: Unwind Your Mind,273,TV-G,2021
2491,The School of Mischief,253,TV-14,1973
2487,No Longer kids,237,TV-14,1979
2484,Lock Your Girls In,233,TV-PG,1982
2488,Raya and Sakina,230,TV-14,1984
166,Once Upon a Time in America,229,R,1984
7932,Sangam,228,TV-14,1964
1019,Lagaan,224,PG,2001
4573,Jodhaa Akbar,214,TV-14,2008




### Pregunta Desafío

11. ¿Cuáles son las combinaciones más frecuentes de género y rating en el dataset?
    (Sugerencia: utilizar `value_counts` con `subset=["genre", "rating"]` después de aplicar `explode()`).



### Bonus: Análisis de duplicados y limpieza

12. ¿Existen películas con el mismo nombre (`title`) pero con distinto año de lanzamiento (`release_year`)?
13. ¿Cuántos títulos únicos hay en total en la columna `title`?





In [21]:
#pregunta desafio
combos_genero_rating = (
    df_generos
      .value_counts(subset=["genre", "rating"])
      .rename("conteo")
      .reset_index()
      .head(10)
)
print(combos_genero_rating)

#bonus 12
# Quedarse solo con películas y los campos relevantes
movies = df.loc[df["type"] == "Movie", ["title", "release_year"]]

# Considerar cada par (title, release_year) una sola vez
pairs = movies.drop_duplicates()

# Contar cuántos años distintos tiene cada título
counts = pairs["title"].value_counts()

# Títulos que aparecen en más de un año (=> mismo nombre con distintos años)
titles_multi_year = counts[counts > 1]

print("¿Existen? ->", not titles_multi_year.empty)
print("N° de títulos con más de un año:", titles_multi_year.size)
 #bonus 13
 # 13) ¿Cuántos títulos únicos hay en 'title'?
titulos_unicos = df["title"].nunique()              # excluye NaN (por defecto)
print("Títulos únicos (sin NaN):", titulos_unicos)


                    genre rating  conteo
0    International Movies  TV-MA    1130
1    International Movies  TV-14    1065
2                  Dramas  TV-MA     830
3  International TV Shows  TV-MA     714
4                  Dramas  TV-14     693
5  International TV Shows  TV-14     472
6                Comedies  TV-14     465
7               TV Dramas  TV-MA     434
8                Comedies  TV-MA     431
9                  Dramas      R     375
¿Existen? -> False
N° de títulos con más de un año: 0
Títulos únicos (sin NaN): 8807
